# Gradient-based optimization

In [33]:
import drjit as dr
import mitsuba as mi

mi.set_variant('llvm_ad_rgb')

In [34]:
dr.__file__

'/Users/yannizhang/Desktop/RGL/mitsuba3/build/python/drjit/__init__.py'

In [35]:
# scene = mi.load_file('scenes/cbox.xml', integrator='prb')
scene = mi.load_file('scenes/checkerboard.xml')




### Reference image

In [36]:
image_ref = mi.render(scene)

# Preview the reference image
mi.util.convert_to_bitmap(image_ref)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [37]:
params = mi.traverse(scene)
params

SceneParameters[
  ----------------------------------------------------------------------------------------
  Name                                 Flags    Type  Parent
  ----------------------------------------------------------------------------------------
  camera.near_clip                              float PerspectiveCamera
  camera.far_clip                               float PerspectiveCamera
  camera.shutter_open                           float PerspectiveCamera
  camera.shutter_open_time                      float PerspectiveCamera
  camera.film.size                              ScalarVector2u HDRFilm
  camera.film.crop_size                         ScalarVector2u HDRFilm
  camera.film.crop_offset                       ScalarPoint2u HDRFilm
  camera.x_fov                         ∂, D     Float PerspectiveCamera
  camera.to_world                      ∂, D     Transform4f PerspectiveCamera
  emitter-envmap.scale                 ∂        Float EnvironmentMapEmitter
  emitter-envm

### Initial state

In [38]:
# key = 'red.reflectance.value'
# key = 'white.reflectance.value'
# key = 'DiffuseBSDF.brdf_0.reflectance.value'
key = 'texture-checkerboard.color1.value'


# Save the original value
param_ref = mi.Color3f(params[key])

# Set another color value and update the scene
params[key] = mi.Color3f(0.01, 0.2, 0.9)
params.update();

As expected, when rendering the scene again, the wall has changed color.

In [39]:
image_init = mi.render(scene)
mi.util.convert_to_bitmap(image_init)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

## Optimization

For gradient-based optimization, Mitsuba ships with standard optimizers including *Stochastic Gradient Descent* ([<code>SGD</code>][1]) with and without momentum, as well as [<code>Adam</code>][2] <cite data-cite="kingma2014adam">[KB14]</cite>. We will instantiate the latter and optimize our scene parameter with a learning rate of `0.05`. 

We then set the color to optimize on the optimizer, which will now hold a copy of this parameter and enable gradient tracking on it. During the optimization process, the optimizer will always perfom gradient steps on those variables. To propagate those changes to the scene, we need to call the `update()` method which will copy the values back into the `params` data structure. As always this method also notifies all objects in the scene whose parameters have changed, in case they need to update their internal state.

This first call to `params.update()` ensures that gradient tracking with respect to our wall color parameter is propagated to the scene internal state. For more detailed explanation on how-to-use the optimizer classes, please refer to the dedicated [how-to-guide][3].

[1]: https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.SGD
[2]: https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.Adam
[3]: https://mitsuba.readthedocs.io/en/latest/src/how_to_guides/use_optimizers.html

In [40]:
opt = mi.ad.Adam(lr=0.05)
# opt = mi.ad.AdamVar(lr=0.05)
# opt = mi.ad.AdamVarC(lr=0.05)


opt[key] = params[key]
params.update(opt);

At every iteration of the gradient descent, we will compute the derivatives of the scene parameters with respect to the objective function. In this simple experiment, we use the [*mean square error*][1], or $L_2$ error, between the current image and the reference created above.

[1]: https://en.wikipedia.org/wiki/Mean_squared_error

In [41]:
def mse(image):
    return dr.mean(dr.sqr(image - image_ref))

In the following cell we define the hyper parameters controlling our optimization loop, such as the number of iterations:

In [42]:
iteration_count = 50

In [43]:
# IGNORE THIS: When running under pytest, adjust parameters to reduce computation time
import os
if 'PYTEST_CURRENT_TEST' in os.environ:
    iteration_count = 2

It is now time to actually perform the gradient-descent loop that executes 50 differentiable rendering iterations.

In [44]:
errors = []
err_init = dr.sum(dr.sqr(param_ref - params[key]))
errors.append(err_init)
for it in range(iteration_count):
    # Perform a (noisy) differentiable rendering of the scene
    image = mi.render(scene, params,spp=1)

    # Evaluate the objective function from the current rendered image
    loss = mse(image)

    # Backpropagate through the rendering process
    dr.backward(loss, flags = dr.ADFlag.BackPropVarianceCounter | dr.ADFlag.ClearVertices)

    # g = dr.graphviz_ad()
    # g.view()

    # Optimizer: take a gradient descent step
    opt.step()

    # Post-process the optimized parameters to ensure legal color values.
    opt[key] = dr.clamp(opt[key], 0.0, 1.0)

    # Update the scene state to the new optimized values
    params.update(opt)
    
    # Track the difference between the current color and the true value
    err_ref = dr.sum(dr.sqr(param_ref - params[key]))
    print(f"Iteration {it:02d}: parameter error = {err_ref[0]:6f}", end='\r')
    errors.append(err_ref)
print('\nOptimization complete.')

before: accum v size 196608; grad size 0; grad2 size 0
after: accum v size 196608; grad size 196608; grad2 size 196608
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65536; grad size 65536; grad2 size 65536
before: accum v size 65536; grad size 0; grad2 size 0
after: accum v size 65

## Results

We can now render the scene again to check whether the optimization process successfully recovered the color of the red wall.

In [45]:
image_final = mi.render(scene, spp=128)
# mi.util.write_bitmap('/Users/yannizhang/Desktop/RGL/5.10/render2_adam_V_C.png', image_final)
mi.util.convert_to_bitmap(image_final)


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

It worked!

Note visualizing the objective value directly sometimes gives limited information, since differences between `image` and `image_ref` can be dominated by Monte Carlo noise that is not related to the parameter being optimized. 

Since we know the “true” target parameter in this scene, we can validate the convergence of the optimization by checking the difference to the true color at each iteration:

In [46]:
import matplotlib.pyplot as plt
plt.plot(errors)
plt.xlabel('Iteration'); plt.ylabel('MSE(param)'); plt.title('Parameter error plot');
# plt.savefig('/Users/yannizhang/Desktop/RGL/5.10/scene2_adma_V_C.png')
plt.show()

## See also

- [Detailed look at <code>Optimizer</code>](https://mitsuba.readthedocs.io/en/latest/src/how_to_guides/use_optimizers.html)
- API reference:
  - [<code>mitsuba.ad.Optimizer</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.Optimizer)
  - [<code>mitsuba.ad.integrators.prb</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.integrators.prb)
  - [<code>mitsuba.ad.SGD</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.SGD)
  - [<code>mitsuba.ad.Adam</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.Adam)
  - [<code>drjit.backward</code>](https://drjit.readthedocs.io/en/latest/src/api_reference.html#drjit.backward)